https://github.com/quantopian/zipline/issues/911


```
conda create -n py27 python=2.7 anaconda
source activate py27
conda install -c Quantopian zipline=1.1.1
conda install pandas-datareader==0.2.1
conda install networkx==1.9.1
pip install ipykernel
python -m ipykernel install --user --name py27 --display-name "Python (py27)"
zipline ingest

pip install alphalens
jupyter notebook
```

In [1]:
from zipline.data import bundles
from zipline.pipeline import Pipeline
from zipline.pipeline.data import USEquityPricing
from zipline.pipeline.data import Column  
from zipline.pipeline.data import DataSet
from zipline.pipeline.engine import SimplePipelineEngine
from zipline.pipeline.filters import StaticAssets
from zipline.pipeline.loaders import USEquityPricingLoader
from zipline.pipeline.loaders.frame import DataFrameLoader
from zipline.utils.calendars import get_calendar

import numpy as np
import pandas as pd

trading_calendar = get_calendar('NYSE')
bundle_data = bundles.load('quandl')


# Set up Custom Data Source for two sids for DataFrameLoader
class MyDataSet(DataSet):  
    column_A = Column(dtype=float)
    column_B = Column(dtype=bool) 

dates = pd.date_range('2014-01-01', '2017-01-01', tz='UTC')
assets = bundle_data.asset_finder.lookup_symbols(['A', 'AAL'], as_of_date=None)
sids = pd.Int64Index([asset.sid for asset in assets])

# The values for Column A will just be a 2D array of numbers ranging from 1 -> N.  
column_A_frame = pd.DataFrame(  
    data=np.arange(len(dates)*len(assets), dtype=float).reshape(len(dates), len(assets)),  
    index=dates,
    columns=sids,
)

# Column B will always provide True for 0 and False for 1.  
column_B_frame = pd.DataFrame(data={sids[0]: True, sids[1]: False}, index=dates)

loaders = {  
    MyDataSet.column_A: DataFrameLoader(MyDataSet.column_A, column_A_frame),  
    MyDataSet.column_B: DataFrameLoader(MyDataSet.column_B, column_B_frame),  
}

def my_dispatcher(column):
    return loaders[column]


# Set up pipeline engine

# Loader for pricing
pipeline_loader = USEquityPricingLoader(
    bundle_data.equity_daily_bar_reader,
    bundle_data.adjustment_reader,
)

def choose_loader(column):
    if column in USEquityPricing.columns:
        return pipeline_loader
    return my_dispatcher(column)

engine = SimplePipelineEngine(
    get_loader=choose_loader,
    calendar=trading_calendar.all_sessions,
    asset_finder=bundle_data.asset_finder,
)

p = Pipeline(
    columns={
        'price': USEquityPricing.close.latest,
        'col_A': MyDataSet.column_A.latest,
        'col_B': MyDataSet.column_B.latest
    },
    screen=StaticAssets(assets)
)

df = engine.run_pipeline(
    p,
    pd.Timestamp('2016-01-05', tz='utc'),
    pd.Timestamp('2016-01-07', tz='utc')
)

print df

                                            col_A  col_B  price
2016-01-05 00:00:00+00:00 Equity(0 [A])    1468.0   True  40.69
                          Equity(2 [AAL])  1469.0  False  40.91
2016-01-06 00:00:00+00:00 Equity(0 [A])    1470.0   True  40.55
                          Equity(2 [AAL])  1471.0  False  40.52
2016-01-07 00:00:00+00:00 Equity(0 [A])    1472.0   True  40.73
                          Equity(2 [AAL])  1473.0  False  41.23


In [2]:
print np.__version__

1.11.3


In [3]:
loader = my_dispatcher(MyDataSet.column_A)
adj_array = loader.load_adjusted_array(
    [MyDataSet.column_A],
    dates,
    sids,
    np.ones((len(dates), len(sids)), dtype=bool)
)
print adj_array.values()[0].inspect()

Adjusted Array (float64):

Data:
array([[  0.00000000e+00,   1.00000000e+00],
       [  2.00000000e+00,   3.00000000e+00],
       [  4.00000000e+00,   5.00000000e+00],
       ..., 
       [  2.18800000e+03,   2.18900000e+03],
       [  2.19000000e+03,   2.19100000e+03],
       [  2.19200000e+03,   2.19300000e+03]])

Adjustments:
{}



In [4]:
type(MyDataSet.column_A)

zipline.pipeline.data.dataset.BoundColumn

In [5]:
column_A_frame.columns

Int64Index([0, 2], dtype='int64')